# Basic being Bayesian by backpropagation

In [1]:
import nbloader
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
from demo_correlatedFields import fields
from demo_util import nzr,gpusession
from demo_tf_sample import bbb
print ("Packages Loaded")

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Importing Jupyter notebook from [demo_correlatedFields.ipynb]
Importing Jupyter notebook from [demo_util.ipynb]
Importing Jupyter notebook from [demo_tf_sample.ipynb]
Packages loaded
TF version: 1.3.0
Packages Loaded


Construct graph

In [2]:
class bayesianNetwork(object):
    def __init__(self,_xdim=2,_ydim=1,_hdims=[32,32],_actv=tf.nn.relu,
                _bn=slim.batch_norm,_sess=None):
        # Parse inputs
        self.xdim = _xdim
        self.ydim = _ydim
        self.hdims = _hdims
        self.actv = _actv
        self.bn = _bn
        self.sess = _sess
        # Initialize graph
        self.init_randomWeights()
        self.construct_graph()
        
    def init_randomWeights(self):
        self.Qweights = []
        self.Qbiases = []
        self.Pweights = []
        self.Pbiases = []
        # Input to first hidden layer
        self.Qweights.append(bbb(_name='Qw%d'%0,_distribution='Gaussian',
                                 _shape=[self.xdim,self.hdims[0]],_trainable=True))
        self.Qbiases.append(bbb(_name='Qb%d'%0,_distribution='Gaussian',
                                 _shape=[self.hdims[0]],_trainable=True))
        self.Pweights.append(bbb(_name='Pw%d'%0,_distribution='Gaussian',
                                 _shape=[self.xdim,self.hdims[0]],_trainable=False))
        self.Pbiases.append(bbb(_name='Pb%d'%0,_distribution='Gaussian',
                                 _shape=[self.hdims[0]],_trainable=False))
        # Hidden to hidden 
        for hIdx in range(len(self.hdims)-1):
            hdim = self.hdims[hIdx]
            self.Qweights.append(bbb(_name='Qw%d'%(hIdx+1),_distribution='Gaussian',
                                     _shape=[self.hdims[hIdx],self.hdims[hIdx+1]],
                                     _trainable=True))
            self.Qbiases.append(bbb(_name='Qb%d'%(hIdx+1),_distribution='Gaussian',
                                     _shape=[self.hdims[hIdx+1]],
                                     _trainable=True))
            self.Pweights.append(bbb(_name='Pw%d'%(hIdx+1),_distribution='Gaussian',
                                     _shape=[self.hdims[hIdx],self.hdims[hIdx+1]],
                                     _trainable=False))
            self.Pbiases.append(bbb(_name='Pb%d'%(hIdx+1),_distribution='Gaussian',
                                     _shape=[self.hdims[hIdx+1]],
                                     _trainable=False))
        # Hidden to output
        nLayer = len(self.hdims)+1
        self.Qweights.append(bbb(_name='Qw%d'%nLayer,_distribution='Gaussian',
                                 _shape=[self.hdims[-1],self.ydim],_trainable=True))
        self.Qbiases.append(bbb(_name='Qb%d'%nLayer,_distribution='Gaussian',
                                 _shape=[self.ydim],_trainable=True))
        self.Pweights.append(bbb(_name='Pw%d'%nLayer,_distribution='Gaussian',
                                 _shape=[self.hdims[-1],self.ydim],_trainable=False))
        self.Pbiases.append(bbb(_name='Pb%d'%nLayer,_distribution='Gaussian',
                                 _shape=[self.ydim],_trainable=False))
    def construct_graph(self):
        self.x = tf.placeholder(dtype=tf.float32,shape=[None,self.xdim],name='Input')
        self.y = tf.placeholder(dtype=tf.float32,shape=[None,self.ydim],name='Output')
        self.nx = tf.shape(self.x)[0] # number of inputs
        
    # Stochastic forward path 
    def stochastic_forwardPath(self,_x):
        # Input to first hidden layer
        nw = len(self.Qweights) # number of weights
        nx = _x.shape[0] # number of inputs (number of samples)
        # Sample weights and biases
        h,w = nw,nx;
        sampledWs = [['' for x in range(w)] for y in range(h)] 
        sampledBs = [['' for x in range(w)] for y in range(h)] 
        for wIdx in range(nw): # For all layer weights
            cQw = self.Qweights[wIdx]
            cQb = self.Qbiases[wIdx]
            sampledW = self.sess.run(cQw.sampler,feed_dict={cQw.nSample:nx})
            sampledB = self.sess.run(cQb.sampler,feed_dict={cQb.nSample:nx})
            for sIdx in range(nx): # For all samples (=number of inputs)
                sampledWs[wIdx][sIdx] = np.reshape(sampledW[sIdx,:],cQw.shape)
                sampledBs[wIdx][sIdx] = np.reshape(sampledB[sIdx,:],cQb.shape)
                print wIdx,sIdx,sampledWs[wIdx][sIdx].shape,sampledBs[wIdx][sIdx].shape
        # Compute stochastic forward paths using sampled weights
        
        print nw, nx
        print sampledWs
        
        
print ("Class defined")

Class defined


In [3]:
sess = gpusession()
BN = bayesianNetwork(_xdim=2,_ydim=1,_hdims=[32,32],_actv=tf.nn.relu,
                     _bn=slim.batch_norm,_sess=sess)
sess.run(tf.global_variables_initializer())

bbb[Qw0] instantiated.
Global variables
 [0] Name:[Qw0/Qw0_mu:0] Shape:[[1, 64]]
 [1] Name:[Qw0/Qw0_logSigma:0] Shape:[[1, 64]]
bbb[Qb0] instantiated.
Global variables
 [0] Name:[Qb0/Qb0_mu:0] Shape:[[1, 32]]
 [1] Name:[Qb0/Qb0_logSigma:0] Shape:[[1, 32]]
bbb[Pw0] instantiated.
Global variables
bbb[Pb0] instantiated.
Global variables
bbb[Qw1] instantiated.
Global variables
 [0] Name:[Qw1/Qw1_mu:0] Shape:[[1, 1024]]
 [1] Name:[Qw1/Qw1_logSigma:0] Shape:[[1, 1024]]
bbb[Qb1] instantiated.
Global variables
 [0] Name:[Qb1/Qb1_mu:0] Shape:[[1, 32]]
 [1] Name:[Qb1/Qb1_logSigma:0] Shape:[[1, 32]]
bbb[Pw1] instantiated.
Global variables
bbb[Pb1] instantiated.
Global variables
bbb[Qw3] instantiated.
Global variables
 [0] Name:[Qw3/Qw3_mu:0] Shape:[[1, 32]]
 [1] Name:[Qw3/Qw3_logSigma:0] Shape:[[1, 32]]
bbb[Qb3] instantiated.
Global variables
 [0] Name:[Qb3/Qb3_mu:0] Shape:[[1, 1]]
 [1] Name:[Qb3/Qb3_logSigma:0] Shape:[[1, 1]]
bbb[Pw3] instantiated.
Global variables
bbb[Pb3] instantiated.
Global 

In [4]:
BN.stochastic_forwardPath(_x=np.zeros((10,2)))

0 0 (2, 32) (32,)
0 1 (2, 32) (32,)
0 2 (2, 32) (32,)
0 3 (2, 32) (32,)
0 4 (2, 32) (32,)
0 5 (2, 32) (32,)
0 6 (2, 32) (32,)
0 7 (2, 32) (32,)
0 8 (2, 32) (32,)
0 9 (2, 32) (32,)
1 0 (32, 32) (32,)
1 1 (32, 32) (32,)
1 2 (32, 32) (32,)
1 3 (32, 32) (32,)
1 4 (32, 32) (32,)
1 5 (32, 32) (32,)
1 6 (32, 32) (32,)
1 7 (32, 32) (32,)
1 8 (32, 32) (32,)
1 9 (32, 32) (32,)
2 0 (32, 1) (1,)
2 1 (32, 1) (1,)
2 2 (32, 1) (1,)
2 3 (32, 1) (1,)
2 4 (32, 1) (1,)
2 5 (32, 1) (1,)
2 6 (32, 1) (1,)
2 7 (32, 1) (1,)
2 8 (32, 1) (1,)
2 9 (32, 1) (1,)
3 10
[[array([[ 1.2607652 , -0.31207955, -1.0193775 ,  1.0180411 ,  0.39107347,
        -0.4584157 , -0.16346279,  0.82609105, -0.4054571 ,  0.18467802,
         1.2303367 , -0.55711335,  1.3549708 ,  0.61795926, -0.03259003,
         0.8477751 ,  0.1088438 , -0.34678182,  0.3729415 , -0.6199666 ,
        -0.23259771,  1.187807  ,  2.4008515 , -2.4322984 , -0.8527973 ,
         0.5120174 ,  1.0425348 , -0.41250503,  0.16164139,  0.25166708,
         0.5462

In [5]:
x.shape[0]

NameError: name 'x' is not defined

In [ ]:
h,w = 8,5;
Matrix = [['' for x in range(w)] for y in range(h)] 
print Matrix
print Matrix[7][4]
print Matrix[4][7]

In [ ]:
Matrix[0][0] = [1,2,3,4,]
print Matrix
print len(Matrix)

Make correlated field

In [ ]:
F = fields(_name='fields',_VERBOSE=False,
           _axisInfo=np.array([0,10,0,10]),
           _resFineInfo=np.array([100,100]),
           _resCoarseInfo=np.array([20,20]),
           _nField=10)
levs = np.array([[1,1,1,1,0,0,0,-1,-1,-1]]).astype(np.float).T
hyp = {'gain':1,'len':1/2.,'s2w':1e-10}
F.genCorrelatedFields(_levs=levs,_rseed=0,_hyp=hyp)
F.getMeasurements(_nMeas=50,_rseed=0)
F.getAccumulatedMeas()
# Use first field
firstField = F.fields[0]
firstField.plotField(_figsize=(7,7),_title='firstField',
                     _PLOT_COARSE=False,_PLOT_MEAS=True)
X = firstField.xyFineMeas
Y = firstField.zFineMeas
print 'X.shape:',X.shape
print 'Y.shape:',Y.shape